In [1]:
import ee
import geemap

In [2]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import ipyleaflet

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
        print(f"Error initializing Earth Engine: {e}")
        exit()

# Create a map centered on Florida
Map = geemap.Map(center=[28.0, -82.4], zoom=7)  # Approximate center of Florida

# Load the Landsat 5 Collection 2 Tier 1-2 surface reflectance data
landsat5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filterDate('1985-01-01', '2000-12-31')  # Filter for the entire period

# Function to apply scaling factors
def applyScaleFactors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
        return image.addBands(opticalBands, None, True) \
            .addBands(thermalBand, None, True)

landsat5 = landsat5.map(applyScaleFactors)

# Function to mask clouds using the pixel_qa band
def maskL5sr(image):
        qaMask = image.select('QA_PIXEL').bitwiseAnd(0b1111).eq(0)
        saturationMask = image.select('QA_RADSAT').eq(0)
        return image.updateMask(qaMask).updateMask(saturationMask)

landsat5_masked = landsat5.map(maskL5sr)

# Load Florida state boundary
florida = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Florida'))

# Define visualization parameters for the thermal band
thermal_vis = {
        'min': 270,  # Example minimum temperature in Kelvin
        'max': 310,  # Example maximum temperature in Kelvin
        'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}


# Function to create and add annual thermal composites
def get_annual_thermal_composite(year):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year, 12, 31)  # Full year
        annual_collection = landsat5_masked.filterDate(start_date, end_date)
        thermal_composite = annual_collection.select('ST_B6').median().clip(florida)
        return thermal_composite


# Create a list of years
years = list(range(1985, 2001))  # 1985 to 2000, inclusive

# Create a dropdown widget
year_dropdown = widgets.Dropdown(
        options=years,
        value=years[0],  # Default to the first year
        description='Select Year:',
        disabled=False,
)

# Function to update the map based on the selected year
def update_map(change):
    selected_year = change.new
    thermal_composite = get_annual_thermal_composite(selected_year)
    thermal_layer_name = f'L5 Thermal {selected_year} (Florida)'

    # Check if the layer already exists and remove it
    layer_to_remove = None
    for layer in Map.layers:
        if layer.name.startswith('L5 Thermal'): # Check for layers added by this function
            layer_to_remove = layer
            break
    if layer_to_remove:
        Map.remove_layer(layer_to_remove)


    # Add the thermal layer
    Map.add_layer(thermal_composite, thermal_vis, thermal_layer_name)
    # add_legend() # Legend only needs to be added once

# Function to add the legend with temperature values in a colorbar style
def add_legend():
    # Get the color palette and temperature range
    palette = thermal_vis['palette']
    min_temp = thermal_vis['min']
    max_temp = thermal_vis['max']
    num_colors = len(palette)

    # Create the HTML for the legend
    legend_html = "<div style='padding: 10px; background-color: white; border: 1px solid #ccc; border-radius: 5px;'>" \
                  "<h4 style='margin-bottom: 5px;'>Surface Temperature (K)</h4>" \
                  "<div style='display: flex; align-items: center;'>"

    # Add color tiles
    for color in palette:
        legend_html += f"<div style='width: 30px; height: 20px; background-color: {color};'></div>"

    legend_html += "</div>"  # Close color bar div

    # Add temperature labels
    legend_html += "<div style='display: flex; justify-content: space-between; margin-top: 5px;'>" \
                    f"<div>{min_temp:.2f}</div>"  # Minimum temperature
    # Add intermediate labels if desired, for simplicity we just add min and max
    legend_html += f"<div>{max_temp:.2f}</div>"  # Maximum temperature
    legend_html += "</div>"  # Close temperature labels div
    legend_html += "</div>"  # Close legend container div

    # Create an HTML widget to display the legend
    # CORRECTED: Use ipywidgets.HTML instead of ipyleaflet.Element
    legend_widget = widgets.HTML(value=legend_html)

    # Create a WidgetControl to add the HTML widget to the map
    legend_control = ipyleaflet.WidgetControl(widget=legend_widget, position='bottomright')

    # Remove any existing legends added by this function to avoid duplicates
    controls_to_remove = []
    for control in Map.controls:
         # Check if the control is a WidgetControl and its widget is an HTML widget likely from the legend
        if isinstance(control, ipyleaflet.WidgetControl) and isinstance(control.widget, widgets.HTML):
             # A more robust check might inspect the content of the HTML if there were other HTML widgets
             controls_to_remove.append(control)

    for control in controls_to_remove:
        Map.remove_control(control)


    # Add the legend to the map
    Map.add_control(legend_control)


# Observe the dropdown changes
year_dropdown.observe(update_map, names='value')

# Add Florida boundary to the map (add this only once)
Map.addLayer(florida, {'color': 'white'}, 'Florida Boundary', False)

# Add the initial thermal layer for the default year
initial_year = year_dropdown.value
initial_thermal_composite = get_annual_thermal_composite(initial_year)
Map.add_layer(initial_thermal_composite, thermal_vis, f'L5 Thermal {initial_year} (Florida)')

# Add the legend
add_legend()

# Display the dropdown and the map
display(year_dropdown)
Map

Dropdown(description='Select Year:', options=(1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995…

Map(center=[28.0, -82.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [3]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import ipyleaflet

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    print(f"Error initializing Earth Engine: {e}")
    exit()

# --- Configuration ---
START_YEAR = 1985
END_YEAR = 2024
TARGET_REGION = 'Florida'
MAP_CENTER = [28.0, -82.4]  # Approximate center of Florida
MAP_ZOOM = 7

# --- Load Florida boundary ---
try:
    region_geom = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', TARGET_REGION)).first().geometry()
except Exception as e:
    print(f"Could not load boundary for {TARGET_REGION}: {e}. Using map center.")
    region_geom = ee.Geometry.Point(MAP_CENTER).buffer(300000)

# --- Visualization Parameters (°C) ---
thermal_vis = {
    'min': -3,
    'max': 37,
    'palette': ['000080', '0000FF', '00FFFF', '00FF00', 'FFFF00', 'FFA500', 'FF0000']
}

# --- Annual Composite Function ---
def get_annual_thermal_composite_for_year(year):
    collection_id = None
    thermal_band = None
    sensor_name = None
    sensor_short_name = None

    if 1985 <= year <= 2011:
        collection_id = 'LANDSAT/LT05/C02/T1_L2'
        thermal_band = 'ST_B6'
        sensor_short_name = 'L5'
    elif 2012 <= year <= 2013:
        collection_id = 'LANDSAT/LE07/C02/T1_L2'
        thermal_band = 'ST_B6'
        sensor_short_name = 'L7'
    elif 2014 <= year <= 2022:
        collection_id = 'LANDSAT/LC08/C02/T1_L2'
        thermal_band = 'ST_B10'
        sensor_short_name = 'L8'
    elif 2023 <= year <= END_YEAR:
        collection_id = 'LANDSAT/LC09/C02/T1_L2'
        thermal_band = 'ST_B10'
        sensor_short_name = 'L9'
    else:
        return None, "Invalid Year"

    def applyScale(image):
        # Convert thermal band from Kelvin to Celsius
        thermal = image.select(thermal_band).multiply(0.00341802).add(149.0).subtract(273.15).rename('surface_temp')
        return image.addBands(thermal, None, True)

    def applyMask(image):
        qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11000', 2)).eq(0)
        saturationMask = image.select('QA_RADSAT').eq(0)
        return image.updateMask(qaMask).updateMask(saturationMask)

    try:
        collection = ee.ImageCollection(collection_id)
        annual_collection = collection.filterBounds(region_geom) \
                                      .filterDate(f'{year}-01-01', f'{year}-12-31') \
                                      .map(applyScale) \
                                      .map(applyMask)

        if annual_collection.size().getInfo() == 0:
            return None, f"{sensor_short_name} (No images)"

        thermal_composite = annual_collection.select('surface_temp').median().clip(region_geom).selfMask()
        return thermal_composite, sensor_short_name

    except Exception as e:
        print(f"Error processing year {year}: {e}")
        return None, f"{sensor_short_name} (Error)"

# --- Map and UI Setup ---
Map = geemap.Map(center=MAP_CENTER, zoom=MAP_ZOOM)
years = list(range(START_YEAR, END_YEAR + 1))
year_dropdown = widgets.Dropdown(options=years, value=END_YEAR, description='Select Year:')
sensor_info_label = widgets.Label(value="Sensor info will appear here...")
current_thermal_layer = None
current_layer_name = ""

def update_map(change):
    global current_thermal_layer, current_layer_name
    selected_year = change.new
    sensor_info_label.value = f"Requesting data for {selected_year}..."

    if current_thermal_layer is not None:
        for layer in Map.layers:
            if layer.name == current_layer_name:
                try:
                    Map.remove_layer(layer)
                except Exception as e:
                    print(f"Error removing layer: {e}")
        current_thermal_layer = None
        current_layer_name = ""

    thermal_composite, sensor_status = get_annual_thermal_composite_for_year(selected_year)
    sensor_info_label.value = f"Year: {selected_year}, Source: {sensor_status}"

    if thermal_composite is not None and "Error" not in sensor_status and "Invalid" not in sensor_status:
        layer_name = f"Thermal {selected_year} ({sensor_status})"
        try:
            Map.add_layer(thermal_composite, thermal_vis, layer_name)
            current_layer_name = layer_name
            for layer in Map.layers:
                if layer.name == layer_name:
                    current_thermal_layer = layer
                    break
        except Exception as e:
            print(f"Error adding layer: {e}")

# --- Legend Setup ---
legend_control = None
def add_legend():
    global legend_control
    if legend_control is not None and legend_control in Map.controls:
        try:
            Map.remove_control(legend_control)
        except:
            pass

    palette = thermal_vis['palette']
    min_temp = thermal_vis['min']
    max_temp = thermal_vis['max']
    num_colors = len(palette)
    step = (max_temp - min_temp) / num_colors

    legend_html = "<div style='padding:10px;background:rgba(255,255,255,0.8);border-radius:5px;'>"
    legend_html += "<b>Surface Temp (°C)</b><br>"
    for i, color in enumerate(palette):
        label = f"{min_temp + i * step:.1f} – {min_temp + (i + 1) * step:.1f}"
        if i == 0:
            label = f"≤ {min_temp + step:.1f}"
        if i == num_colors - 1:
            label = f"> {min_temp + i * step:.1f}"
        legend_html += f"<div style='display:flex;align-items:center;'><div style='width:20px;height:15px;background:#{color};margin-right:5px;border:1px solid #eee;'></div>{label}</div>"
    legend_html += "</div>"

    legend_widget = widgets.HTML(value=legend_html)
    legend_control = ipyleaflet.WidgetControl(widget=legend_widget, position='bottomright')
    Map.add_control(legend_control)

# --- Initialize ---
year_dropdown.observe(update_map, names='value')
Map.addLayer(region_geom, {'color': 'white', 'fillColor': '00000000'}, f'{TARGET_REGION} Boundary', True)

# Add initial layer
initial_year = year_dropdown.value
sensor_info_label.value = f"Loading initial map for {initial_year}..."
initial_composite, initial_sensor = get_annual_thermal_composite_for_year(initial_year)
if initial_composite:
    name = f'Thermal {initial_year} ({initial_sensor})'
    Map.add_layer(initial_composite, thermal_vis, name)
    current_layer_name = name
    for layer in Map.layers:
        if layer.name == name:
            current_thermal_layer = layer
            break
sensor_info_label.value = f"Year: {initial_year}, Source: {initial_sensor}"

add_legend()

# Display UI
display(widgets.VBox([year_dropdown, sensor_info_label]))
display(Map)


Map(center=[28.0, -82.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [4]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from IPython.display import display

# -----------------------------------------------------------------------------
# Authenticate & Initialize
# -----------------------------------------------------------------------------
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
FLORIDA = ee.FeatureCollection("TIGER/2018/States") \
             .filter(ee.Filter.eq('NAME', 'Florida')) \
             .first().geometry()

LST_YEARS = list(range(1985, 2025))
NLCD_COLLECTION = ee.ImageCollection("projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER")
NLCD_YEARS = NLCD_COLLECTION.aggregate_array("system:index").distinct().sort().getInfo()

NLCD_VALUES = [11,12,21,22,23,24,31,41,42,43,52,71,81,82,90,95]
NLCD_COLORS = [
    '466b9f','d1def8','dec5c5','d99282','eb0000','ab0000','b3ac9f','68ab5f',
    '1c5f2c','b5c58f','ccb879','dfdfc2','dcd939','ab6c28','b8d9eb','6c9fb8'
]
NLCD_LABELS = [
    'Open Water','Perennial Ice/Snow','Dev, Open Space','Dev, Low Intensity',
    'Dev, Medium Intensity','Dev, High Intensity','Barren Land','Deciduous Forest',
    'Evergreen Forest','Mixed Forest','Shrub/Scrub','Grassland/Herbaceous',
    'Pasture/Hay','Cultivated Crops','Woody Wetlands','Emerg. Herb. Wetlands'
]

thermal_vis = {'min': -3, 'max': 37, 'palette': ['000080','0000FF','00FFFF','00FF00','FFFF00','FFA500','FF0000']}
utfvi_vis = {'min': -0.3, 'max': 0.3, 'palette': ['0000FF','FFFFFF','FF0000']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['00000000','FF0000']}
ndvi_vis = {'min': -0.2, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401']}
ndbi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['FFFFFF', 'B4B4B4', '999999', '7F7F7F', '666666', '4C4C4C', '333333']}

# -----------------------------------------------------------------------------
# Functions
# -----------------------------------------------------------------------------
def get_annual_lst(year):
    if 1985 <= year <= 2011:
        coll_id, band = 'LANDSAT/LT05/C02/T1_L2', 'ST_B6'
    elif 2012 <= year <= 2013:
        coll_id, band = 'LANDSAT/LE07/C02/T1_L2', 'ST_B6'
    elif 2014 <= year <= 2022:
        coll_id, band = 'LANDSAT/LC08/C02/T1_L2', 'ST_B10'
    else:
        coll_id, band = 'LANDSAT/LC09/C02/T1_L2', 'ST_B10'

    def to_celsius(img):
        return img.select(band).multiply(0.00341802).add(149.0).subtract(273.15).rename('surface_temp')

    def mask(img):
        qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
        sat = img.select('QA_RADSAT').eq(0)
        return img.updateMask(qa).updateMask(sat)

    return ee.ImageCollection(coll_id) \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .map(mask).map(to_celsius).median().clip(FLORIDA).selfMask()

def get_ndvi(year):
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31')

    def compute_ndvi(image):
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        return image.addBands(ndvi)

    return collection.map(compute_ndvi).median().select('NDVI').clip(FLORIDA)

def get_ndbi(year):
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31')

    def compute_ndbi(image):
        ndbi = image.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')
        return image.addBands(ndbi)

    return collection.map(compute_ndbi).median().select('NDBI').clip(FLORIDA)

# -----------------------------------------------------------------------------
# Map & Widgets
# -----------------------------------------------------------------------------
m = geemap.Map(center=[28.0, -82.4], zoom=7)
m.addLayer(FLORIDA, {'color': 'white', 'fillColor': '00000000'}, 'Florida')

param_dd = widgets.Dropdown(options=['LST','UTFVI','Hotspots','NDVI','NDBI','LULC'], value='LST', description='Parameter:')
year_dd = widgets.Dropdown(options=LST_YEARS, value=2024, description='Year:')
info_lbl = widgets.Label()
_prev = None

# -----------------------------------------------------------------------------
# Legend
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    for c in m.controls:
        if getattr(c, 'is_legend', False):
            m.remove_control(c)

    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    if title == 'LULC':
        for v, c, l in zip(NLCD_VALUES, NLCD_COLORS, NLCD_LABELS):
            html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:#{c};margin-right:4px;border:1px solid #aaa;'></div>{v}: {l}</div>"
    else:
        palette = vis['palette']
        step = (vis['max'] - vis['min']) / len(palette)
        for i, col in enumerate(palette):
            lo, hi = vis['min'] + i*step, vis['min'] + (i+1)*step
            label = f"≤ {hi:.2f}" if i == 0 else f"> {lo:.2f}" if i == len(palette)-1 else f"{lo:.2f} – {hi:.2f}"
            html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:#{col};margin-right:4px;border:1px solid #aaa;'></div>{label}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    ctrl.is_legend = True
    m.add_control(ctrl)

# -----------------------------------------------------------------------------
# Dynamic Map Update
# -----------------------------------------------------------------------------
def update_map(*_):
    global _prev
    if _prev:
        try: m.remove_layer(_prev)
        except: pass

    p, y = param_dd.value, str(year_dd.value)
    info_lbl.value = f'Loading {p} {y}…'

    if p == 'LST':
        img, vis, name = get_annual_lst(int(y)), thermal_vis, f'LST {y}'
    elif p == 'UTFVI':
        lst = get_annual_lst(int(y))
        mean = ee.Number(lst.reduceRegion(ee.Reducer.mean(), FLORIDA, 1000, bestEffort=True).get('surface_temp'))
        img, vis, name = lst.subtract(mean).divide(mean).rename('UTFVI'), utfvi_vis, f'UTFVI {y}'
    elif p == 'Hotspots':
        lst = get_annual_lst(int(y))
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), True), FLORIDA, 1000, bestEffort=True)
        mean, std = ee.Number(stats.get('surface_temp_mean')), ee.Number(stats.get('surface_temp_stdDev'))
        img = lst.gt(mean.add(std.multiply(2))).rename('hotspot')
        vis, name = hotspot_vis, f'Hotspots {y}'
    elif p == 'NDVI':
        img, vis, name = get_ndvi(int(y)), ndvi_vis, f'NDVI {y}'
    elif p == 'NDBI':
        img, vis, name = get_ndbi(int(y)), ndbi_vis, f'NDBI {y}'
    else:
        img = NLCD_COLLECTION.filter(ee.Filter.eq('system:index', y)).first().select('b1').clip(FLORIDA)
        vis, name = {'min':11, 'max':95, 'palette':NLCD_COLORS}, f'LULC {y}'

    _prev = m.add_layer(img, vis, name)
    add_legend(vis, 'LULC' if p=='LULC' else p)
    info_lbl.value = f'{p} {y} loaded'

# Param dropdown observer

def on_param_change(change):
    if change.new == 'LULC':
        year_dd.options = NLCD_YEARS
        year_dd.value = NLCD_YEARS[-1]
    else:
        year_dd.options = LST_YEARS
        year_dd.value = max(LST_YEARS)
    update_map()

param_dd.observe(on_param_change, names='value')
year_dd.observe(update_map, names='value')

# -----------------------------------------------------------------------------
# Display
# -----------------------------------------------------------------------------
display(widgets.HBox([param_dd, year_dd]), info_lbl)
display(m)
update_map()

Label(value='')

Map(center=[28.0, -82.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [5]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# -----------------------------------------------------------------------------
# Authenticate & Initialize
# -----------------------------------------------------------------------------
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
FLORIDA = ee.FeatureCollection("TIGER/2018/States") \
             .filter(ee.Filter.eq('NAME', 'Florida')) \
             .first().geometry()

YEARS = list(range(1985, 2024))
NLCD_COLLECTION = ee.ImageCollection("projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER")
NLCD_YEARS = NLCD_COLLECTION.aggregate_array("system:index").distinct().sort().getInfo()

thermal_vis = {'min': -3, 'max': 37, 'palette': ['000080','0000FF','00FFFF','00FF00','FFFF00','FFA500','FF0000']}
utfvi_vis = {'min': -0.3, 'max': 0.3, 'palette': ['0000FF','FFFFFF','FF0000']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['00000000','FF0000']}
ndvi_vis = {'min': -0.2, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401']}
ndbi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['FFFFFF', 'B4B4B4', '999999', '7F7F7F', '666666', '4C4C4C', '333333']}

# -----------------------------------------------------------------------------
# Functions to Get Images
# -----------------------------------------------------------------------------
def get_annual_lst(year):
    if 1985 <= year <= 2011:
        coll_id, band = 'LANDSAT/LT05/C02/T1_L2', 'ST_B6'
    elif 2012 <= year <= 2013:
        coll_id, band = 'LANDSAT/LE07/C02/T1_L2', 'ST_B6'
    elif 2014 <= year <= 2022:
        coll_id, band = 'LANDSAT/LC08/C02/T1_L2', 'ST_B10'
    else:
        coll_id, band = 'LANDSAT/LC09/C02/T1_L2', 'ST_B10'

    def to_celsius(img):
        return img.select(band).multiply(0.00341802).add(149.0).subtract(273.15).rename('LST')

    def mask(img):
        qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
        sat = img.select('QA_RADSAT').eq(0)
        return img.updateMask(qa).updateMask(sat)

    return ee.ImageCollection(coll_id) \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .map(mask).map(to_celsius).median().clip(FLORIDA)

def get_ndvi(year):
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31')
    def compute_ndvi(image):
        return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return collection.map(compute_ndvi).median().clip(FLORIDA)

def get_ndbi(year):
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31')
    def compute_ndbi(image):
        return image.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')
    return collection.map(compute_ndbi).median().clip(FLORIDA)

def compute_stat_image(image, reducer=ee.Reducer.mean()):
    return image.reduceRegion(reducer=reducer, geometry=FLORIDA, scale=1000, bestEffort=True)

# -----------------------------------------------------------------------------
# Parameter Dropdowns and Action
# -----------------------------------------------------------------------------
PARAMS = ['LST', 'NDVI', 'NDBI']
param1_dd = widgets.Dropdown(options=PARAMS, description='Parameter 1:')
param2_dd = widgets.Dropdown(options=PARAMS, description='Parameter 2:')
year_dd = widgets.Dropdown(options=YEARS, description='Year:')
analysis_dd = widgets.Dropdown(options=['Line Graph', 'Correlation'], description='Analysis:')
plot_btn = widgets.Button(description='Plot')
output = widgets.Output()

# -----------------------------------------------------------------------------
# Plotting Function
# -----------------------------------------------------------------------------
def fetch_param_image(param, year):
    if param == 'LST':
        return get_annual_lst(year).select('LST')
    elif param == 'NDVI':
        return get_ndvi(year).select('NDVI')
    elif param == 'NDBI':
        return get_ndbi(year).select('NDBI')
    else:
        return None

@output.capture(clear_output=True)
def on_plot_click(b):
    p1, p2, selected_year, method = param1_dd.value, param2_dd.value, year_dd.value, analysis_dd.value

    if method == 'Line Graph':
        y_values_1 = []
        y_values_2 = []
        for yr in YEARS:
            img1 = fetch_param_image(p1, yr)
            img2 = fetch_param_image(p2, yr)
            try:
                val1 = img1.reduceRegion(ee.Reducer.mean(), FLORIDA, 1000, bestEffort=True).getInfo()[p1]
                val2 = img2.reduceRegion(ee.Reducer.mean(), FLORIDA, 1000, bestEffort=True).getInfo()[p2]
                y_values_1.append(val1)
                y_values_2.append(val2)
            except:
                y_values_1.append(None)
                y_values_2.append(None)

        fig, ax1 = plt.subplots()
        ax1.plot(YEARS, y_values_1, label=p1, color='tab:blue')
        ax1.set_ylabel(p1, color='tab:blue')
        ax1.tick_params(axis='y', labelcolor='tab:blue')

        ax2 = ax1.twinx()
        ax2.plot(YEARS, y_values_2, label=p2, color='tab:green')
        ax2.set_ylabel(p2, color='tab:green')
        ax2.tick_params(axis='y', labelcolor='tab:green')

        plt.title(f'{p1} vs {p2} ({YEARS[0]}–{YEARS[-1]})')
        plt.show()

    elif method == 'Correlation':
        img1 = fetch_param_image(p1, selected_year)
        img2 = fetch_param_image(p2, selected_year)
        sample = img1.addBands(img2).sample(region=FLORIDA, scale=1000, numPixels=5000, geometries=False).getInfo()

        x = [s['properties'][p1] for s in sample['features'] if p1 in s['properties'] and p2 in s['properties']]
        y = [s['properties'][p2] for s in sample['features'] if p1 in s['properties'] and p2 in s['properties']]

        if x and y:
            plt.scatter(x, y, alpha=0.5)
            plt.xlabel(p1)
            plt.ylabel(p2)
            plt.title(f'{p1} vs {p2} Correlation ({selected_year})')
            plt.grid(True)
            plt.show()
        else:
            print("Not enough valid data points for correlation plot.")

plot_btn.on_click(on_plot_click)

# -----------------------------------------------------------------------------
# Display UI
# -----------------------------------------------------------------------------
display(widgets.VBox([widgets.HBox([param1_dd, param2_dd]), year_dd, analysis_dd, plot_btn, output]))


In [6]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from IPython.display import display
import os
import datetime

# Authenticate & Initialize
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# Configuration
FLORIDA = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Florida')).first().geometry()
LST_YEARS = list(range(1985, 2025))

thermal_vis = {'min': -3, 'max': 37, 'palette': ['000080','0000FF','00FFFF','00FF00','FFFF00','FFA500','FF0000']}
ndvi_vis = {'min': -0.2, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401']}
ndbi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['FFFFFF', 'B4B4B4', '999999', '7F7F7F', '666666', '4C4C4C', '333333']}

# Functions
def get_annual_lst(year):
    if 1985 <= year <= 2011:
        coll_id, band = 'LANDSAT/LT05/C02/T1_L2', 'ST_B6'
    elif 2012 <= year <= 2013:
        coll_id, band = 'LANDSAT/LE07/C02/T1_L2', 'ST_B6'
    elif 2014 <= year <= 2022:
        coll_id, band = 'LANDSAT/LC08/C02/T1_L2', 'ST_B10'
    else:
        coll_id, band = 'LANDSAT/LC09/C02/T1_L2', 'ST_B10'

    def to_celsius(img):
        return img.select(band).multiply(0.00341802).add(149.0).subtract(273.15).rename('surface_temp')

    def mask(img):
        qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
        sat = img.select('QA_RADSAT').eq(0)
        return img.updateMask(qa).updateMask(sat)

    return ee.ImageCollection(coll_id) \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .map(mask).map(to_celsius).median().clip(FLORIDA).selfMask()

def get_ndvi(year):
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31')

    def compute_ndvi(image):
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        return image.addBands(ndvi)

    return collection.map(compute_ndvi).median().select('NDVI').clip(FLORIDA)

def get_ndbi(year):
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(FLORIDA) \
        .filterDate(f'{year}-01-01', f'{year}-12-31')

    def compute_ndbi(image):
        ndbi = image.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')
        return image.addBands(ndbi)

    return collection.map(compute_ndbi).median().select('NDBI').clip(FLORIDA)

# Animation Creation
def create_animation(parameter, start_year, end_year, out_gif):
    vis_params = {
        'LST': thermal_vis,
        'NDVI': ndvi_vis,
        'NDBI': ndbi_vis,
    }

    def get_image(year):
        if parameter == 'LST':
            return get_annual_lst(year).visualize(**thermal_vis).set({'label': str(year)})
        elif parameter == 'NDVI':
            return get_ndvi(year).visualize(**ndvi_vis).set({'label': str(year)})
        elif parameter == 'NDBI':
            return get_ndbi(year).visualize(**ndbi_vis).set({'label': str(year)})
        else:
            raise ValueError("Animation only supported for LST, NDVI, and NDBI.")

    images = [get_image(y) for y in range(start_year, end_year + 1)]
    collection = ee.ImageCollection(images)

    video_args = {
        'dimensions': 768,
        'region': FLORIDA,
        'framesPerSecond': 2,
        'crs': 'EPSG:3857',
        'min': 0,
        'max': 255
    }

    geemap.download_ee_video(collection, video_args, out_gif)

# Map Setup
m = geemap.Map(center=[28.0, -82.4], zoom=7)
m.addLayer(FLORIDA, {'color': 'white', 'fillColor': '00000000'}, 'Florida')

# Widgets
param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI'], value='LST', description='Parameter:')
start_year_slider = widgets.IntSlider(value=2000, min=1985, max=2024, step=1, description='Start Year:')
end_year_slider = widgets.IntSlider(value=2024, min=1985, max=2024, step=1, description='End Year:')
info_lbl = widgets.Label()
anim_button = widgets.Button(description='Download Animation', button_style='success')

# Event: Download Animation
def on_anim_button_click(b):
    param = param_dd.value
    start = start_year_slider.value
    end = end_year_slider.value

    if start > end:
        info_lbl.value = '⚠️ Start year must be ≤ End year.'
        return

    info_lbl.value = f'Creating animation for {param} from {start} to {end}...'
    now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    out_gif = f'{param}_animation_{start}_{end}_{now}.gif'

    try:
        create_animation(param, start, end, out_gif)
        info_lbl.value = f'✅ Animation saved as {out_gif}'
    except Exception as e:
        info_lbl.value = f'❌ Error: {e}'

anim_button.on_click(on_anim_button_click)

# Display Widgets
display(widgets.VBox([param_dd, widgets.HBox([start_year_slider, end_year_slider]), anim_button, info_lbl]))
display(m)


Map(center=[28.0, -82.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…